In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl

from astropy.io import fits
from astropy.wcs import WCS
from kcwitools.io import open_kcwi_cube
from kcwitools.utils import build_wave
from kcwitools import extract_weighted_spectrum as ke
from kcwitools import image as im
from kcwitools import utils as kcwi_u

import warnings
warnings.filterwarnings('ignore')

def transform_pixels(x1, y1, wcs1, wcs2):
    r, d = wcs1.pixel_to_world_values(x1, y1)
    x2, y2 = wcs2.world_to_pixel_values(r, d)
    return x2, y2
def find_fits(basedir, endswith):
    return (os.path.join(root, file)
        for root, dirs, files in os.walk(basedir)
            for file in files if root == basedir and file.lower().endswith(endswith))
def find_files_ending_with(directory, endswith):
    # List all files in the specified directory
    files = os.listdir(directory)
    # Filter the list to only include files that end with the specified suffix
    matching_files = [file for file in files if file.endswith(endswith)]
    return matching_files
def find_fits(basedir, endswith):
    return (
        os.path.join(root, file)
        for root, dirs, files in os.walk(basedir)
        for file in files if root == basedir and file.lower().endswith(endswith)
        )
def find_index_exceeding_value(lst, val):
    indices = [index for index, value in enumerate(lst) if value > val]
    return indices[0] if indices else None


In [ ]:
def read_and_prep_flux_var_data(flux_file, var_file, nb_index_lo, nb_index_hi):
    """ This method reads the flux and var cubes for a specific observation, and cleans them up before returning """
    hdr_f, flux = open_kcwi_cube(flux_file)
    
    _, var = open_kcwi_cube(var_file)
    wave = build_wave(hdr_f)
    # low_index = find_index_exceeding_value(wave, wl_low)
    # high_index = find_index_exceeding_value(wave, wl_high)
    low_index = nb_index_lo
    high_index = nb_index_hi

    # first do a little data cleanup
    var[np.isnan(flux)]=1.
    flux[np.isnan(flux)]=0.0000

    print("="*50)
    print(f"wave[{low_index}]:{wave[low_index]}  wave[{high_index}]:{wave[high_index]}")
    # wave = wave[low_index:high_index]
    # flux = flux[low_index:high_index]
    # var = var[low_index:high_index]
    print(f"wave[0]:{wave[0]}  wave[-1]:{wave[-1]}")
    print("="*50)
    # wcs_f = WCS(hdr_f).celestial

    # Create a narrow-band whitelight image to plot
    whitelight= im.build_whitelight(hdr_f, flux, minwave=wave[0], maxwave=wave[-1])
    return flux, var, wave, whitelight, hdr_f


In [ ]:
def label_axes(fig):
    for i, ax in enumerate(fig.axes):
        ax.text(0.5, 0.5, "ax%d" % (i+1), va="center", ha="center")
        ax.tick_params(labelbottom=False, labelleft=False)

In [2]:
def create_images_layout(plt, count=6):
    """returns an arround of {count} axes for images to be placed within"""
    fig = plt.figure(figsize=(14,10), layout="constrained")
    # fig.suptitle(title)
    axs_image = []
    gs = gridspec.GridSpec(10, count, figure=fig)
    for i in range(count):
        print(f"index:{i}")
        axs_image.append(fig.add_subplot(gs[0:9, i]))
        label_axes(fig)
    # ax_spectrum = fig.add_subplot(gs[10:12, :])
    # return fig, ax_spectrum, axs_image
    return fig, axs_image

In [ ]:
def unused1():
    flux_files = list(sorted(find_fits(basedir,"_corrected_flux.fits")))
    var_files = list(sorted(find_fits(basedir,"_corrected_var.fits")))
    # zipped = zip(flux_files, var_files)
    data = list(zip(flux_files, var_files))
    print(f"We have {len(data)} files to process")
    for ii, datum in enumerate(data):
        print(f"data[{ii}]: {datum[0]}:{datum[1]}")


In [ ]:
def display_wl_image(ax, title, wl_image, point, hdr_f):
    xc = [point[0]]
    yc = [point[1]]
    lw = 0.5
    box_size_arcsecond = 2

    cmap = 'gnuplot'
    # cmap = 'grey'
    # valid values for cmap; supported values are 'Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r', 'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Grays', 'Greens', 'Greens_r', 'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r', 'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r', 'Pastel2', 'Pastel2_r', 'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r', 'PuBu_r', 'PuOr', 'PuOr_r', 'PuRd', 'PuRd_r', 'Purples', 'Purples_r', 'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r', 'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r', 'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r', 'Set1', 'Set1_r', 'Set2', 'Set2_r', 'Set3', 'Set3_r', 'Spectral', 'Spectral_r', 'Wistia', 'Wistia_r', 'YlGn', 'YlGnBu', 'YlGnBu_r', 'YlGn_r', 'YlOrBr', 'YlOrBr_r', 'YlOrRd', 'YlOrRd_r', 'afmhot', 'afmhot_r', 'autumn', 'autumn_r', 'binary', 'binary_r', 'bone', 'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cividis', 'cividis_r', 'cool', 'cool_r', 'coolwarm', 'coolwarm_r', 'copper', 'copper_r', 'cubehelix', 'cubehelix_r', 'flag', 'flag_r', 'gist_earth', 'gist_earth_r', 'gist_gray', 'gist_gray_r', 'gist_grey', 'gist_heat', 'gist_heat_r', 'gist_ncar', 'gist_ncar_r', 'gist_rainbow', 'gist_rainbow_r', 'gist_stern', 'gist_stern_r', 'gist_yarg', 'gist_yarg_r', 'gist_yerg', 'gnuplot', 'gnuplot2', 'gnuplot2_r', 'gnuplot_r', 'gray', 'gray_r', 'grey', 'hot', 'hot_r', 'hsv', 'hsv_r', 'inferno', 'inferno_r', 'jet', 'jet_r', 'magma', 'magma_r', 'nipy_spectral', 'nipy_spectral_r', 'ocean', 'ocean_r', 'pink', 'pink_r', 'plasma', 'plasma_r', 'prism', 'prism_r', 'rainbow', 'rainbow_r', 'seismic', 'seismic_r', 'spring', 'spring_r', 'summer', 'summer_r', 'tab10', 'tab10_r', 'tab20', 'tab20_r', 'tab20b', 'tab20b_r', 'tab20c', 'tab20c_r', 'terrain', 'terrain_r', 'turbo', 'turbo_r', 'twilight', 'twilight_r', 'twilight_shifted', 'twilight_shifted_r', 'viridis', 'viridis_r', 'winter', 'winter_r'
    # playing around with the color map
    print(f"wl_image.shape={wl_image.shape}")
    # trying to figure out a good range for the color map
    # wl_min = np.amin(wl_image)
    # wl_max = np.amax(wl_image)
    # print(f"wl_min={wl_min}   wl_max={wl_max}")
    im1 = ax.imshow(wl_image, origin='lower', interpolation='nearest', cmap=cmap, vmin=0, vmax=80)
    #plt.colorbar(im1)
    ax.plot(xc, yc, 'gs')
    ax.grid()
    box_size = int(box_size_arcsecond / (hdr_f["CD2_2"] * 3600))
    hb = box_size // 2
    ax.set_xlabel("RA")
    ax.set_ylabel("Decl.")
    ax.set_title(title, fontstyle='oblique', fontsize='medium')
    for i in range(len(xc)):
        ax.plot([xc[i] - hb, xc[i] + hb, xc[i] + hb, xc[i] - hb, xc[i] - hb], 
                [yc[i] - hb, yc[i] - hb, yc[i] + hb, yc[i] + hb, yc[i] - hb], 'w-', lw=lw)
    # fig.suptitle(k_fits_list[l][:24])


In [ ]:
configs = {
    "base_dir":"/Users/robertseaton/rwseaton@ncsu.edu - Google Drive/My Drive/Astro Research/KCWI_1429_Data/Fluxed",
    "observations":[
        {
            "flux":"KB.20180708.23055_icubes_corrected_flux.fits",
            "var":"KB.20180708.23055_icubes_corrected_var.fits",
            "pt_x": 17, 
            "pt_y": 39,
            "nb_wave_lo": 4645.0,
            "nb_wave_hi": 4659.0,
            "nb_index_lo": 1418,
            "nb_index_hi": 1432,
            "window_size_x": 0,
            "window_size_y": 0,
            "image_shift_x": 0,
            "image_shift_y": 0
        },
        {
            "flux":"KB.20180708.24349_icubes_corrected_flux.fits",
            "var":"KB.20180708.24349_icubes_corrected_var.fits",
            "pt_x": 16, 
            "pt_y": 43,
            "nb_index_lo": 1420,
            "nb_index_hi": 1434,
            "nb_wave_lo": 4646.02,
            "nb_wave_hi`": 4660.08,
            "window_size_x": 0,
            "window_size_y": 0,
            "image_shift_x": 0,
            "image_shift_y": 0
        },
        {
            "flux":"KB.20190601.32100_icubes_corrected_flux.fits",
            "var":"KB.20190601.32100_icubes_corrected_var.fits",
            "pt_x": 18,
            "pt_y": 36,
            "nb_index_lo": 1420,
            "nb_index_hi": 1431,
            "nb_wave_lo": 4646.26,
            "nb_wave_hi`": 4657.01,
            "window_size_x": 0,
            "window_size_y": 0,
            "image_shift_x": 0,
            "image_shift_y": 0
        },
        {
            "flux":"KB.20190601.33358_icubes_corrected_flux.fits",
            "var":"KB.20190601.33358_icubes_corrected_var.fits",
            "pt_x": 18, 
            "pt_y": 36,
            "nb_index_lo": 1419,
            "nb_index_hi": 1435,
            "nb_wave_lo": 4645.07,
            "nb_wave_hi`": 4661.1,
            "window_size_x": 0,
            "window_size_y": 0,
            "image_shift_x": 0,
            "image_shift_y": 0
        },
        {
            "flux":"KB.20200520.37578_icubes_corrected_flux.fits",
            "var":"KB.20200520.37578_icubes_corrected_var.fits",
            "pt_x": 17, 
            "pt_y": 39,
            "nb_index_lo": 1418,
            "nb_index_hi": 1433,
            "nb_wave_lo": 4644.62,
            "nb_wave_hi`": 4660.42,
            "window_size_x": 0,
            "window_size_y": 0,
            "image_shift_x": 0,
            "image_shift_y": 0
        },
        {
            "flux":"KB.20200520.38847_icubes_corrected_flux.fits",
            "var":"KB.20200520.38847_icubes_corrected_var.fits",
            "pt_x": 17, 
            "pt_y": 39,
            "nb_index_lo": 1418,
            "nb_index_hi": 1434,
            "nb_wave_lo": 4645.45,
            "nb_wave_hi`": 4660.39,
            "window_size_x": 0,
            "window_size_y": 0,
            "image_shift_x": 0,
            "image_shift_y": 0
        },
        {
            "flux":"KB.20200520.40119_icubes_corrected_flux.fits",
            "var":"KB.20200520.40119_icubes_corrected_var.fits",
            "pt_x": 17, 
            "pt_y": 39,
            "nb_index_lo": 1416,
            "nb_index_hi": 1432,
            "nb_wave_lo": 4643.4,
            "nb_wave_hi`": 4658.92,
            "window_size_x": 0,
            "window_size_y": 0,
            "image_shift_x": 0,
            "image_shift_y": 0
        }
    ]
}

In [3]:
def display_test():

    count = 7
    fig, axs_image = create_images_layout(plt, count=count)
    fig.suptitle(f"{count} observations of 1429")
    plt.show()
display_test()

NameError: name 'plt' is not defined

In [ ]:
def main_ish():
    basedir = configs["base_dir"]
    flux_files = list(sorted(find_fits(basedir,"_corrected_flux.fits")))
    var_files = list(sorted(find_fits(basedir,"_corrected_var.fits")))
    zipped = zip(flux_files, var_files)
    print(repr(enumerate(zipped)))

    wl_low = 3500
    wl_high = 5500 
    
    spectrum_sum = None # this needs to have the same shape as the specturm shape
    variance_sum = None # this needs to have the same the specturm shape
    axs = []
    data = list(zip(flux_files, var_files))
    count = len(data)
    fig, axs_image = create_images_layout(plt, count=count)
    fig.suptitle(f"{count} observations of 1429")
    observations = configs["observations"]
    

    for ii, datum in enumerate(data):
        o = observations[ii]
        ffile = datum[0]
        vfile = datum[1]
        nb_index_lo = o["nb_index_lo"]-15
        nb_index_hi = o["nb_index_hi"]+15
        flux, var, wave, wl_image, hdr_f = read_and_prep_flux_var_data(ffile, vfile, nb_index_lo, nb_index_hi)

        print("- "*20)
        # set up the layout
        # ax_image, ax_spec, ax_variance = create_image_flux_var_layout(plt, title="GridSpec Layout")
        # axs.append((ax_image, ax_spec, ax_variance))
        point=[o["pt_x"],o["pt_y"]]
        # display the WL image
        title = datum[0].split("/")[-1].split("_icubes_corrected_flux.fits")[0]
        display_wl_image(axs_image[ii], title, wl_image, point, hdr_f)

        # with warnings.catch_warnings():
        #     # Ignore model linearity warning from the fitter
        #     warnings.simplefilter('ignore')
        #     sp=ke.extract_weighted_spectrum(flux, var, wave, weights='Data', verbose=False)
        
        # display the point or points on the WL image
        # extract the weighted spectrum and variance
        # display the spectrum and variance
        # add spectrum / variance to 
    plt.show()


main_ish()
